### Loading dataset

In [28]:
from google.colab import drive
from google.colab import files
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to dataset file on Google Drive
folder_path2014 = "/content/drive/MyDrive/SEM 06/6SP/Raw Operational Data from Enterprise Application - KAGGLE.zip (Unzipped Files)/2014"
folder_path2015 = "/content/drive/MyDrive/SEM 06/6SP/Raw Operational Data from Enterprise Application - KAGGLE.zip (Unzipped Files)/2015"

folders = [folder_path2014, folder_path2015]

# Log file list
log_files = []

file_path = "/content/drive/MyDrive/SEM 06/6SP/all_log_files.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
  for folder in folders:
    for dirname, _, filenames in os.walk(folder):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            # Write content to the file
            file.write(os.path.join(dirname, filename))
            file.write("\n")


print("All file names saved to all_log_files.txt ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All file names saved to all_log_files.txt 


In [29]:
import random

# Set the range for random numbers
start_range = 1
end_range = 18337
num_samples = 10

# Generate a list of random numbers
random_numbers = random.sample(range(start_range, end_range + 1), num_samples)

# Print the list of random numbers
print(random_numbers)

[16030, 2498, 228, 1358, 12898, 11977, 3515, 9022, 11331, 9424]


### Creating dataframes from log files


In [43]:
import pandas as pd

# Read the CSV file into a DataFrame
# df = pd.read_csv(file_path)
# print(df)

# List of dataframes
dfs = []


# file_path = "/content/drive/MyDrive/SEM 06/6SP/all_log_files.txt"

i = 0 # For line number

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the file line by line
    for line in file:
      i += 1
      if i > max(random_numbers):
        break
      if i in random_numbers:
        dfs.append( pd.read_csv(line.strip(),low_memory=False))


print(len(dfs))


10


###Dropping empty log file dataframes

In [44]:
print("Initial no. of dataframes = ",len(dfs))
empty_file_count = 0
i = -1
for df in dfs:
  i +=1
  if len(df) == 0:
    empty_file_count +=1
    print("Empty file found! Total = ",empty_file_count)
    del dfs[i]


print("Final no. of dataframes = ",len(dfs))

Initial no. of dataframes =  10
Empty file found! Total =  1
Final no. of dataframes =  9


### Viewing some df from dfs

In [46]:
len(dfs[3])
dfs[3].head()

,Date,\Process(java#1)\RES,\Process(java#1)\USER,\Process(java#1)\SHR,\Process(java#1)\PR,\Process(java#1)\S,\Process(java#1)\CPU,\Process(java#1)\NI,\Process(java#1)\PID,\Process(java#1)\MEM,\Process(java#1)\TIME,\Process(java#1)\VIRT,Unnamed: 12
0,04.12.2014 00:00:43.913,"1,595,115","553,650,281","32,505,856",20,83,0,0,"9,704",14.8,"125,933","46,817,760",NaN
1,04.12.2014 00:01:44.904,"1,595,115","553,650,281","32,505,856",20,83,0,0,"9,704",14.8,"125,934","46,817,760",NaN
2,04.12.2014 00:02:46.012,"1,595,115","553,650,281","32,505,856",20,83,2,0,"9,704",14.8,"125,935","46,817,760",NaN
3,04.12.2014 00:03:47.151,"1,595,115","553,650,281","32,505,856",20,83,0,0,"9,704",14.8,"125,936","46,817,760",NaN
4,04.12.2014 00:04:48.527,"1,595,115","553,650,281","32,505,856",20,83,2,0,"9,704",14.8,"125,938","46,817,760",NaN


### Creating log file clusters based on column names

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


# Load log files into separate DataFrames
dataframes = dfs

# Extract column names from each DataFrame
column_names = []
for df in dataframes:
    column_names.append(df.columns.tolist())

print(len(column_names[3]))



13


In [58]:
# Creating a pyton set to store column names
columns_set =set()

file_path = "/content/drive/MyDrive/SEM 06/6SP/columns.txt"
# Open the file in write mode
with open(file_path, 'w') as file:
  for cols in column_names:
    for col in cols:
      file.write(col)
      file.write('\n')

with open(file_path, 'r') as file:
    lines = file.readlines()


# Reading column names and adding to the set
with open(file_path, 'r') as file:
    for line in file:
      columns_set.add(line)
# lines = all columns including repetitions
print('Duplicate coumn names = ',len(lines)-len(columns_set))



# Create feature matrix
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(lines)

# Apply dimensionality reduction
pca = PCA(n_components=2)  # You can adjust the number of components
X_reduced = pca.fit_transform(X.toarray())

# Perform clustering with DBSCAN
scaler = StandardScaler()
X_reduced_scaled = scaler.fit_transform(X_reduced)




Duplicate coumn names =  436


In [26]:
dbscan = DBSCAN(eps=0.5, min_samples=2)  # Adjust the eps and min_samples values <----------------------------------
clusters = dbscan.fit_predict(X_reduced_scaled)

print(clusters)


# Assign log files to clusters
log_files_clusters = {}
for i, file in enumerate(log_files):
    log_files_clusters[file] = clusters[i]

# Print the log file and its corresponding cluster
for file, cluster in log_files_clusters.items():
    print(f"Log file: {file}, Cluster: {cluster}")
    print("Done!")


print("Done!!!")

[0 0 0 ... 0 0 0]
Done!!!
